##  Let me try [Parrelle is all you want](https://github.com/IliaZenkov/transformer-cnn-emotion-recognition/blob/main/notebooks/Parallel_is_All_You_Want.ipynb), but only adaopting its model.
 21 Oct 2020

In [ ]:
!git clone https://github.com/fastai/course-v3/ /content/course-v3
%cd /content/course-v3/nbs/dl2

- NOTE: if you do not have installed [nvidia/apex](https://github.com/nvidia/apex),  change `from exp.nb_10c import *` to `from exp.nb_10b import *` in `exp.nb_11`

In [6]:
from exp.nb_12a import *

- Local data (which is divided into separate dataset) is saved to `Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/v1/ser'`


In [8]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [9]:
root_path = Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1'); root_path.ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/dev')]

features-entry contains a list of a list with 26 items.
- length of inner list: 26 (float numbers - represent one preprocessed speech frame (logMel))

- length of outer list: number of frames per data-point, e.g. 10 or 15, ...

In [10]:
train_path = root_path/'train'

In [11]:
audios = get_files(train_path)

In [12]:
# restart-run
class AudioList(ItemList):
    @classmethod
    def from_files(cls, path, extensions = None, recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, fn):
        return torch.load(fn)

In [13]:
al=AudioList.from_files(train_path); al

AudioList (7800 items)
[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7760_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7761_0_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7762_0_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7763_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7764_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7765_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7766_1_1.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7767_1_0.pt'), PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/data/ser/v1/train/7768_1_0.p

## Use adaptive pooling to fit the frames

## input transform

1. load tensor
2. tocuda
3. transpose, add dimension to first axis
4. AdaptiveAvgPooling to fit width(time frames) 250


In [30]:
class ToCuda():
    _order=20
    def __call__(self, ad):
        return ad.cuda()

In [31]:
class Reshape():
    _order=30
    def __call__(self, item):
        w, h = item.shape
        return item.view(h, w).unsqueeze(0)

In [40]:
class FrameAvgPooling():
    _order=40
    def __init__(self, n_tf):
        # n_tf: numner of time frames
        self.p = nn.AdaptiveAvgPool1d(n_tf)
    def __call__(self, item):
        return self.p(item)

In [224]:
class FramePooling():
    _order=40
    def __init__(self, f, n_tf):
        # n_tf: numner of time frames
        self.p = f(n_tf)
    def __call__(self, item):
        return self.p(item)

In [226]:
tfms = [ToCuda(), Reshape(), FramePooling(f= nn.AdaptiveMaxPool1d, n_tf = 150)]
il = AudioList.from_files(train_path, tfms = tfms)

In [227]:
il[0].shape

torch.Size([1, 26, 150])

In [228]:
def re_labeler(fn, pat, subcl='act'):
    assert subcl in ['act', 'val', 'all']
    if subcl=='all': return ''.join(re.findall(pat, str(fn)))
    else:
        return re.findall(pat, str(fn))[0] if pat == 'act' else re.findall(pat, str(fn))[1]

label_pat = r'_(\d+)'
emotion_labeler = partial(re_labeler, pat=label_pat, subcl='all')

In [229]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.2))
ll = label_by_func(sd, emotion_labeler, proc_y=CategoryProcessor())

In [240]:
bs=64
c_in = ll.train[0][0].shape[0]
c_out = len(uniqueify(ll.train.y))
data = ll.to_databunch(bs,c_in=c_in,c_out=c_out)

In [231]:
c_in, c_out

(1, 4)

In [56]:
x, y= ll.train[0]

In [57]:
x.shape, y

(torch.Size([1, 26, 250]), 0)

## Model

In [96]:
#@title
model_demo = nn.Sequential(
    
 nn.Sequential(   nn.Conv2d(
        in_channels=1, # input volume depth == input channel dim == 1
        out_channels=16, # expand output feature map volume's depth to 16
        kernel_size=3, # typical 3*3 stride 1 kernel
        stride=1,
        padding=1
                ),
    nn.BatchNorm2d(16), # batch normalize the output feature map before activation
    nn.ReLU(), # feature map --> activation map
    nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
    nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
    ),
    # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
nn.Sequential(    nn.Conv2d(
        in_channels=16, 
        out_channels=32, # expand output feature map volume's depth to 32
        kernel_size=3,
        stride=1,
        padding=1
                ),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
    nn.Dropout(p=0.3)),
    
    # 3rd 2D convolution layer identical to last except output dim
nn.Sequential(    nn.Conv2d(
        in_channels=32,
        out_channels=64, # expand output feature map volume's depth to 64
        kernel_size=3,
        stride=1,
        padding=1
                ),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.AdaptiveMaxPool2d((1,4)),
    nn.Dropout(p=0.3),)
    )

In [239]:
class Model(nn.Module):
    # Define all layers present in the network
    def __init__(self,num_emotions):
        super().__init__() 
        
        ################ TRANSFORMER BLOCK #############################
        # maxpool the input feature map/tensor to the transformer 
        # a rectangular kernel worked better here for the rectangular input spectrogram feature map/tensor
        self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
        
        # define single transformer encoder layer
        # self-attention + feedforward network from "Attention is All You Need" paper
        # 4 multi-head self-attention layers each with 26-->512--->26 feedforward network
        transformer_layer = nn.TransformerEncoderLayer(
            d_model=26, # input feature (frequency) dim after maxpooling 26*250 -> 26*62 (MFC*time)
            nhead=2, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
            dim_feedforward=256, # 2 linear layers in each encoder block's feedforward network: dim 26-->512--->26
            dropout=0.4, 
            activation='gelu' # ReLU: avoid saturation/tame gradient/reduce compute time
        )
        
        # I'm using 4 instead of the 6 identical stacked encoder layrs used in Attention is All You Need paper
        # Complete transformer block contains 4 full transformer encoder layers (each w/ multihead self-attention+feedforward)
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=4)
        
        ############### 1ST PARALLEL 2D CONVOLUTION BLOCK ############
        # 3 sequential conv2D layers: (1,26,250) --> (16, 13, 125) -> (32, 3, 31) -> (64, 1, 4)
        self.conv2Dblock1 = nn.Sequential(
            
            # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=16, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
            
            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=16, 
                out_channels=32, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3), 
            
            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=32,
                out_channels=64, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((1,4)),
            nn.Dropout(p=0.3),
        )
        ############### 2ND PARALLEL 2D CONVOLUTION BLOCK ############
        # 3 sequential conv2D layers: (1,40,282) --> (16, 20, 141) -> (32, 5, 35) -> (64, 1, 4)
        self.conv2Dblock2 = nn.Sequential(
            
            # 1st 2D convolution layer
            nn.Conv2d(
                in_channels=1, # input volume depth == input channel dim == 1
                out_channels=16, # expand output feature map volume's depth to 16
                kernel_size=3, # typical 3*3 stride 1 kernel
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(16), # batch normalize the output feature map before activation
            nn.ReLU(), # feature map --> activation map
            nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
            nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
            
            # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
            nn.Conv2d(
                in_channels=16, 
                out_channels=32, # expand output feature map volume's depth to 32
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4), # increase maxpool kernel for subsequent filters
            nn.Dropout(p=0.3), 
            
            # 3rd 2D convolution layer identical to last except output dim
            nn.Conv2d(
                in_channels=32,
                out_channels=64, # expand output feature map volume's depth to 64
                kernel_size=3,
                stride=1,
                padding=1
                      ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d((1,4)),
            nn.Dropout(p=0.3),
        )

        ################# FINAL LINEAR BLOCK ####################
        # Linear softmax layer to take final concatenated embedding tensor 
        #    from parallel 2D convolutional and transformer blocks, output 8 logits 
        # Each full convolution block outputs (64*1*8) embedding flattened to dim 512 1D array 
        # Full transformer block outputs 40*70 feature map, which we time-avg to dim 40 1D array
        # 512*2+40 == 1064 input features --> 8 output emotions 
        self.fc1_linear = nn.Linear(256*2+26,num_emotions)
        
        ### Softmax layer for the 8 output logits from final FC linear layer 
        self.softmax_out = nn.Softmax(dim=1) # dim==1 is the freq embedding
        
    # define one complete parallel fwd pass of input feature tensor thru 2*conv+1*transformer blocks
    def forward(self,x):
        
        ############ 1st parallel Conv2D block: 4 Convolutional layers ############################
        # create final feature embedding from 1st convolutional layer 
        # input features pased through 4 sequential 2D convolutional layers
        conv2d_embedding1 = self.conv2Dblock1(x) # x == N/batch * channel * freq * time
        
        # flatten final 64*1*8 feature map from convolutional layers to length 512 1D array 
        # skip the 1st (N/batch) dimension when flattening
        conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim=1) 
        
        ############ 2nd parallel Conv2D block: 4 Convolutional layers #############################
        # create final feature embedding from 2nd convolutional layer 
        # input features pased through 4 sequential 2D convolutional layers
        conv2d_embedding2 = self.conv2Dblock2(x) # x == N/batch * channel * freq * time
        
        # flatten final 64*1*8 feature map from convolutional layers to length 512 1D array 
        # skip the 1st (N/batch) dimension when flattening
        conv2d_embedding2 = torch.flatten(conv2d_embedding2, start_dim=1) 
        
         
        ########## 4-encoder-layer Transformer block w/ 26-->512-->26 feedfwd network ##############
        # maxpool input feature map: 1*40*282 w/ 1*4 kernel --> 1*40*70
        x_maxpool = self.transformer_maxpool(x)

        # remove channel dim: 1*40*70 --> 40*70
        x_maxpool_reduced = torch.squeeze(x_maxpool,1)
        
        # convert maxpooled feature map format: batch * freq * time ---> time * batch * freq format
        # because transformer encoder layer requires tensor in format: time * batch * embedding (freq)
        x = x_maxpool_reduced.permute(2,0,1) 
        
        # finally, pass reduced input feature map x into transformer encoder layers
        transformer_output = self.transformer_encoder(x)
        
        # create final feature emedding from transformer layer by taking mean in the time dimension (now the 0th dim)
        # transformer outputs 2x40 (MFCC embedding*time) feature map, take mean of columns i.e. take time average
        transformer_embedding = torch.mean(transformer_output, dim=0) # dim 40x70 --> 40
        
        ############# concatenate freq embeddings from convolutional and transformer blocks ######
        # concatenate embedding tensors output by parallel 2*conv and 1*transformer blocks
        complete_embedding = torch.cat([conv2d_embedding1, conv2d_embedding2,transformer_embedding], dim=1)  

        ######### final FC linear layer, need logits for loss #########################
        output_logits = self.fc1_linear(complete_embedding)
        
        # need output logits to compute cross entropy loss, need softmax probabilities to predict class
        return output_logits

In [124]:
!pip install -qq torchmetrics
from torchmetrics import F1
f1 = F1(num_classes=4)

     |████████████████████████████████| 332 kB 9.8 MB/s 


In [246]:
# opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)
loss_func = LabelSmoothingCrossEntropy()
lr = 1e-2
pct_start = 0.5
phases = create_phases(pct_start)
sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds(phases, cos_1cycle_anneal(0.95,0.85, 0.95))
cbscheds = [ParamScheduler('lr', sched_lr), 
            ParamScheduler('mom', sched_mom)]

In [234]:
def learner(arch, data, loss_func, opt_func, c_in=None, c_out=None,
                lr=1e-2, cuda=True, norm=None, progress=True, mixup=0, xtra_cb=None, **kwargs):
    cbfs = [partial(AvgStatsCallback,accuracy)]+listify(xtra_cb)
    if progress: cbfs.append(ProgressCallback)
    if cuda:     cbfs.append(CudaCallback)
    if norm:     cbfs.append(partial(BatchTransformXCallback, norm))
    if mixup:    cbfs.append(partial(MixUp, mixup))
    arch_args = {}
    if not c_in : c_in  = data.c_in
    if not c_out: c_out = data.c_out
    if c_in:  arch_args['c_in' ]=c_in
    if c_out: arch_args['c_out']=c_out
    return Learner(arch, data, loss_func, opt_func=opt_func, lr=lr, cb_funcs=cbfs, **kwargs)


In [241]:
model = Model(c_out).cuda()

In [254]:
def sgd_opt(xtra_step=None, **kwargs):
    return partial(Optimizer, steppers=[weight_decay]+listify(xtra_step),
                   stats=[AverageGrad(dampening=True), AverageSqrGrad(), StepCount()], **kwargs)

In [255]:
opt_func = sgd_opt(lr=0.01, weight_decay=1e-3, momentum=0.8)

In [256]:
learn = learner(model, data, loss_func, opt_func=opt_func)

In [257]:
learn.fit(300, cbs=cbscheds)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.520974,0.268680,1.658805,0.289404,00:10
1,2.498805,0.259936,1.663060,0.288079,00:11
2,2.516170,0.267091,1.656659,0.288742,00:11
3,2.515422,0.264388,1.661606,0.288742,00:10
4,2.506654,0.275676,1.659923,0.288742,00:10
5,2.520093,0.267568,1.663013,0.288742,00:10
6,2.508234,0.262957,1.662208,0.288742,00:10
7,2.511315,0.261685,1.652186,0.289404,00:10
8,2.508366,0.269316,1.662082,0.288079,00:10
9,2.508068,0.268362,1.662718,0.288079,00:10


KeyboardInterrupt: ignored

In [192]:
!pip install -qq torchinfo

In [260]:
from torchinfo import summary
summary(Model(4).cuda(), (bs, 1, 26, 150))

Layer (type:depth-idx)                        Output Shape              Param #
Model                                         --                        --
├─TransformerEncoder: 1                       --                        --
│    └─ModuleList: 2-1                        --                        --
├─Sequential: 1-1                             [64, 64, 1, 4]            --
│    └─Conv2d: 2-2                            [64, 16, 26, 150]         160
│    └─BatchNorm2d: 2-3                       [64, 16, 26, 150]         32
│    └─ReLU: 2-4                              [64, 16, 26, 150]         --
│    └─MaxPool2d: 2-5                         [64, 16, 13, 75]          --
│    └─Dropout: 2-6                           [64, 16, 13, 75]          --
│    └─Conv2d: 2-7                            [64, 32, 13, 75]          4,640
│    └─BatchNorm2d: 2-8                       [64, 32, 13, 75]          64
│    └─ReLU: 2-9                              [64, 32, 13, 75]          --
│    └─MaxPool2d

## Save model for inference

In [185]:
%cd /content/

/content


In [258]:
model_path = Path('/content/model/v9'); model_path.mkdir(parents = True, exist_ok=True)

In [259]:
torch.save(learn.model.state_dict(), model_path/'ser-v9-transformer-cnn-mh-2.pt')

In [264]:
!tar -czf model-v5-8.tar.gz model

# References

- [Pytorch Transformer Encoder](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html)
- [fastai course-v3 part2, audio](https://github.com/fastai/course-v3/blob/master/nbs/dl2/audio.ipynb)
- [Transformer-CNN-emotion recognition](https://github.com/IliaZenkov/transformer-cnn-emotion-recognition/blob/main/notebooks/Parallel_is_All_You_Want.ipynb)
- 
